[How to use the python package of correction.](https://gitlab.com/icc-ub/public/gaiadr3_zeropoint) 

[Lindegren 2021: Parallax bias versus magnitude, colour, and position](https://www.aanda.org/articles/aa/full_html/2021/05/aa39653-20/aa39653-20.html)

In [1]:
import numpy as np
import pandas as pd
from astropy.io import ascii

from zero_point import zpt
zpt.load_tables()

In [2]:
help(zpt.get_zpt) 

Help on function get_zpt in module zero_point.zpt:

get_zpt(phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved, _warnings=True)
    Returns the parallax zero point [mas] for a source of given G magnitude, effective wavenumber (nuEff) [1/micron],
    pseudocolour (pc) [1/micron], and ecl_lat [degrees]. It also needs the astrometric_params_solved to discern
    between 5-p and 6-p solutions. Valid for 5- and 6-parameter solutions with 6<G<21 and 1.1<nuEff<1.9,
    1.24<pc<1.72. Outside these ranges, the function can return a very imprecise zero-point.
    
    The inputs can be either floats or an iterable (ndarray, list or tuple). In case of the later, their shape must
    be the same and equal to (N,), where N is the number of sources.
    
    Usage: parallax_corrected = parallax_catalogue - zero_point
    
    Original code: @LL 2020-07-14
    
    NOTE: if any of the inputs values is NaN, the output will be NaN. Also, if the warnings are tur

In [ ]:
data = ascii.read('ngc5822_vizier1.fit')

In [ ]:
#table with the data that you're using from gaiadr3
data = ascii.read('ngc5822_vizier1.fit')

#data used for applying the correction
gmag = data['phot_g_mean_mag'].values
nueffused = data['nu_eff_used_in_astrometry'].values
psc = data['pseudocolour'].values
ecl_lat = data['ecl_lat'].values
soltype = data['astrometric_params_solved'].values

In [ ]:
zpvals = zpt.get_zpt(gmag, nueffused, psc, ecl_lat, soltype)

valid = soltype>3
zpvals = zpt.get_zpt(gmag[valid], nueffused[valid], psc[valid], 
                     ecl_lat[valid], soltype[valid],_warnings=False)

In [ ]:
#adding the value of correction at the end of the table
#this step is not necessary, you can make just the correction and add in the table
data.add_column(zpvals, name='zp_plx_corr')   

In [ ]:
#applying correction in the parallax
plx = data["parallax"] - data["zp_plx_corr"] 

#adding correction in the parallax at the end of the table
data.add_column(plx, name='plx')             